## Analysis of a Stabilization graph
Based on Arie Landau *et al.*, *J. Phys. Chem.* A 2016, 120, 3098−3108

Analytic continuation of a single root using standard Pade approximants and input data from a plateau. 

In [1]:
import numpy as np
from scipy.optimize import minimize_scalar, root_scalar
from scipy import interpolate
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use('Qt5Agg')
%matplotlib qt5
Angs2Bohr=1.8897259886
au2eV=27.211386027
au2cm=219474.63068

In [2]:
#
# put pade.py into the current directory or put its location into the system path
#
import sys
sys.path.append('../../Python_libs')
import pade
import stabtools as st

Landau outlines a ten-step protocol:

### Step 1: Get a real stabilization graph

The scaling parameter $z$ may be a box length $L$  or a scaling factor $\alpha$ of a set of Gaussians.

In [3]:
df = pd.read_csv("DVR_stab_plot.csv")
#df = pd.read_csv("GTO_unc_stab_plot.csv")
#df = pd.read_csv("GTO_DZ_stab_plot.csv")
#df = pd.read_csv("GTO_TZ_stab_plot.csv")

#df=pd.read_csv("/home/thomas/Current_Work/Caroline/Glyoxal/Stab/E_geo0.csv", delim_whitespace=True)
#df=pd.read_csv("/home/thomas/Current_Work/Caroline/Glyoxal/Stab/E_geo-1.csv", delim_whitespace=True)
#df=pd.read_csv("/home/thomas/Current_Work/Caroline/Glyoxal/Stab/Z_geo0_cleaned.csv")
#df=pd.read_csv("/home/thomas/Current_Work/Caroline/Glyoxal/Stab/Z_geo-1_cleaned.csv")
df.head()

,L,z,E1,E2,E3,E4,E5,E6
0,1.423025,0.493827,-7.170508,0.159275,0.523245,1.098066,1.858843,2.728974
1,1.399532,0.510545,-7.170508,0.165750,0.545553,1.144416,1.933952,2.814184
2,1.377166,0.527263,-7.170508,0.172307,0.568155,1.191240,2.009268,2.890628
3,1.355838,0.543981,-7.170508,0.178946,0.591050,1.238525,2.084680,2.956664
4,1.335472,0.560700,-7.170508,0.185668,0.614238,1.286256,2.160057,3.011986


In [4]:
z_col=df.columns[1]
zs=df[z_col].values
zmin, zmax = zs[0], zs[-1]
if zmax < zmin:
    zmin, zmax = zmax, zmin
zs=(zs-zmin)/(zmax-zmin)

E_labels=df.columns[2:]
if True:
    plt.cla()
    for E in E_labels:
        plt.plot(zs, df[E].values, '.')
    plt.ylim(0,8)
    plt.show()

### Step 2: Select the low curvature region of a plateu 

Set the following parameters after inspecting the stabilization plot:
* lroot: root with a stabilization plateau
* if the autoselect doesn't work, narrow search by setting zmin, zmax

In [6]:
xs=zs
lroot=3
ys=df[E_labels[lroot]].values
srch_range=(-1, -1)
drop=0.5
# def find_plateau(xs, ys, srch_range=(-1, -1), drop=0.5, smooth=0, alpha=True):
#     """
#     find the plateau region of one root of a stabilization graph ys(xs)
#     - the left and right crossings are defined be min/max curvature 
#     - the center is min(derivative) or curvature=0
#     Parameters:
#         xs : alphas or Ls
#         ys : Energy of the root
#         srch_range=(xmin, xmax): 
#             search range; (-1,-1) is the whole range, can be used to slice the data 
#         drop: plateau region starts when the curvature drops to this percentage of its max
#         smooth: parameter passed into the spline 
#         alpha: root has positive slope (1/L^2 stabilization graph)
#     Returns: 
#         success, 
#         x_sel, y_sel:
#     """
def der1(x, a, b, c):
    # spline = (a, b, c)
    return interpolate.splev(x, (a,b,c), der=1)

def der2(x, a, b, c):
    # spline = (a, b, c)
    return interpolate.splev(x, (a,b,c), der=2)

def mder2(x, a, b, c):
    # spline = (a, b, c)
    return -interpolate.splev(x, (a,b,c), der=2)

trouble=True
# search range, normally xs[0,len(xs)]
xmin, xmax = srch_range
j0, j1 = 0, len(xs)
if xmin != -1:
    j0 = np.argmin(np.abs(xs-xmin))
if xmax != -1:
    j1 = np.argmin(np.abs(xs-xmax)) + 1
xuse, yuse = xs[j0:j1], ys[j0:j1]
xmin, xmax = xuse[0], xuse[-1]
step=xuse[1]-xuse[0]
print(xmin, xmax)
sp = interpolate.splrep(xuse, yuse, s=0)
d2s = interpolate.splev(xuse, sp, der=2)

# Find the left crossing, xl, from min curvature 
jl=np.argmin(d2s)
xl=xuse[jl]
res = minimize_scalar(der2, (xl-step, xl, xl+step), args=sp)
if res.success:
    xl = ref.x
else:
    trouble = True  
# Find the right crossing, xr, from max curvature 
jr=np.argmax(d2s)
xr=xuse[jr]
res = minimize_scalar(mder2, (xr-step, xr, xr+step), args=sp)
if res.success:
    xr = res.x
else:
    trouble = True

# between xl and xr, find the dropping points d2 = drop*min(d2)    




if trouble:
    plt.cla()
    plt.plot(xuse,d2s,'o-')
    plt.show()

0.0 1.0
0.30833333333333335 -37.103068065958766


0.30833333333333335

In [9]:
lroot=3
zmin, zmax = -1, -1
E_0=df[E_labels[lroot]].values
d1s, d2s = st.der_and_curvature(zs,E_0)
(jc, j1, j2), (zc, z1, z2) = st.plateau(zs,E_0,srch_range=(zmin,zmax))
if jc > 0:
    print('Plateau center at z=%f' % (zc))
if j1 > 0 and j2 > 0:
    print('Crossings at z=%f and z=%f' % (z1, z2))
    if j1 > j2:
        j1, j2 = j2, j1
    d2dn, d2up = d2s[j1], d2s[j2]
    ipdn=j1+np.argmin(np.abs(d2s[j1:j2]-0.5*d2dn))
    ipup=j1+np.argmin(np.abs(d2s[j1:j2]-0.5*d2up))
    alps=zs[ipdn:ipup+1]
    Es=E_0[ipdn:ipup+1]
    npts = len(Es)
    print('N = %d,  max n: %d' % (npts, (npts-1)/2))
    plt.cla()
    plt.plot(zs, E_0, '-')
    plt.plot(alps, Es, 'o')
    plt.ylabel('root '+str(lroot))
    plt.show()
else:
    print('No plateau with adjacent crossings found.')
    print(jc, j1, j2)
    plt.cla()
    plt.plot(zs,d1s,label="1st der")
    plt.plot(zs,d2s,label="2nd der")
    plt.legend()
    plt.ylabel('derivative and curvature')
    plt.show()

Plateau center at z=0.478218
Crossings at z=0.308333 and z=0.666667
N = 23,  max n: 11


### Step 3 and 4: Fit to [$n$,$n$]-Pade approximant and show the result

In [12]:
n=4
P, Q = pade.pade_via_lstsq(n, n, alps, Es)
print('E = P/Q')
print('P :\n ', P)
print('Q :\n ', Q)

#
# compare the input energies and the fitted curves
#
npts=200
aplot=np.linspace(zs[0],zs[-1],npts)
Eplot=P(aplot)/Q(aplot)
#for i in range(npts):
#    E1plot[i]=pade.E_lower(Lplots[i], A, B, C)
#    E2plot[i]=pade.E_upper(Lplots[i], A, B, C)
plt.cla()
plt.plot(zs, E_0, '-.')
plt.plot(alps, Es, 'o')
plt.plot(aplot,Eplot,'-')
plt.ylim(0,6)
plt.show()

E = P/Q
P :
         4         3         2
79.78 x - 162.2 x + 114.5 x - 32.63 x + 3.259
Q :
         4         3         2
25.73 x - 51.63 x + 35.99 x - 10.13 x + 1


### Steps $5-8$

Instead of computing $\theta$ and $\alpha$ trajectories,
* Evaluate the derivative of the Pade approximant in the complex plane.
* Plot its log-abs on a colorscale.
* Read the approximate stable point off the graph.
* Find the precise stable point with Newton-Raphson.

In [13]:
npts=81
rp=1.2 # plus/minus x%
ip=2*zc
Lrs = np.linspace(alps[0]*(1-rp), alps[-1]*(1+rp), npts)
Lis = np.linspace(-ip, 0, npts)
der = np.zeros([npts,npts])
for r in range(npts):
    for i in range(npts):
        L=Lrs[r]+ 1j*Lis[i]
        der[i,r] = np.log10(abs(pade.dEds(L, P, Q)))

plt.cla() 
plt.contourf(Lrs, Lis, der, 10, cmap=plt.cm.jet_r)
plt.colorbar()
plt.show()

In [14]:
#   read-off from graph, derivative and Newton step should be small

#L_guess=0.13+1j*0.33  # DVR
#L_guess=0.40+1j*0.22  # DVR
#L_guess=0.26+1j*0.34  # GTO3
L_guess=0.4-1j*0.2  


der = pade.dEds(L_guess, P, Q)
print("dE/ds =", der)
corr = pade.EpoEpp(L_guess, P, Q)
print("E'/E'' =", corr)

dE/ds = (0.04617419124592006+0.10715286091836371j)
E'/E'' = (-0.005513550321611129+0.025434859797220374j)


In [15]:
# Newton Raphson

max_step = 20
Lstar = L_guess
Ecurr = P(Lstar)/Q(Lstar)
tol=1e-7
for i in range(max_step):
    delta_L = pade.EpoEpp(Lstar, P, Q)
    Lstar = Lstar - delta_L
    Enew = P(Lstar)/Q(Lstar)
    delta_E = Enew - Ecurr
    Ecurr = Enew
    # print table with L E
    print("%3d  (%.7f, %.7f)   (%.7f, %.7f)" % 
          (i, Lstar.real, Lstar.imag, Ecurr.real, Ecurr.imag))
    # check convergence
    if abs(delta_L) < tol and delta_E < tol:
        break
print("\nFinal results:")
print("  L_star = (%.6f, %.6f)" % (Lstar.real, Lstar.imag))
print("  Er = %.6f,  Ei = %.6f" % (Ecurr.real, Ecurr.imag))
# table = (n,Lstar.real, Lstar.imag,Ecurr.real, 2*Ecurr.imag)
# print('| %1d | (%.6f, %.6f) | %.6f | %.6f |' % table)

  0  (0.4055136, -0.2254349)   (3.1737639, -0.1475754)
  1  (0.4070496, -0.2306210)   (3.1738118, -0.1475786)
  2  (0.4071519, -0.2307950)   (3.1738119, -0.1475786)
  3  (0.4071521, -0.2307951)   (3.1738119, -0.1475786)
  4  (0.4071521, -0.2307951)   (3.1738119, -0.1475786)

Final results:
  L_star = (0.407152, -0.230795)
  Er = 3.173812,  Ei = -0.147579


To do **steps 9 and 10**: 
* Go back and repeat with different $n$.
* Use a different percentage than the default.